In [1]:
from dictlearner import DictLearnerFactory
from lif import LIFFactory
import numpy as np
from utils import Config, Log, mkdir
from pallet import ImageLoader
from vis_sc import plot_feedforward_weights, vis_error
config = Config

## Training from scratch

This section can be used to run the training process for an experiment from scratch.

In [6]:
# adjust these constants as desired!
config = Config
config.num_units = 1600
config.num_euler_steps = 50
config.emb_dim = 256
config.WE = 8
config.WI = 8
config.target_l1 = 0.07
config.VT = -0.026

lif_name = "kglight1"
dictlearner_name = "globalposnormregl1"

train_steps = 10

In [7]:
config.fpath = "./result/{}{}/".format(lif_name, dictlearner_name)
mkdir(config.fpath)
logger = Log(config.fpath + "logfile.txt")
config.logger = logger

Initializing loader, LIF, and dictionary learner 

In [8]:
loader = ImageLoader(config.batch_size, config.emb_dim)
lif = LIFFactory.registry[lif_name](config)
lif.init_internal_state()
dictlearner = DictLearnerFactory.registry[dictlearner_name](config, lif)

Training

In [9]:
acts = []
records = np.empty((train_steps, 4))

import time
start_time = time.time()
for t in range(train_steps):
   if t>0 and t%500 == 0:
      print("At step {} out of {}, time elapsed {:.2f}s".format(t, train_steps, time.time()))
      dictlearner.dump(t)
      plot_feedforward_weights(dictlearner.Phi, config.fpath, t)

   batch, _ = loader.load_train_batch()
   try: 
      record = dictlearner.train(batch, t)
      logger.write("At step {} has VT {:.3e} with record".format(t, dictlearner.lif.VT), record)
   except Exception as e:
      print("Error:", e)
      break
   
   acts.append(dictlearner.acts)
   records[t,:] = record

trained_steps = t+1
records = records[:trained_steps]

lif.dump(trained_steps)
dictlearner.dump(trained_steps)
plot_feedforward_weights(dictlearner.Phi, config.fpath, trained_steps)
vis_error(records, config.fpath, 0)